In [1]:
import pylab
import calendar
import numpy as np
import pandas as pd
import seaborn as sn
from scipy import stats
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import kendalltau
import warnings
# from sklearn.linear_model import LinearRegression

matplotlib.style.use('ggplot')
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
ints = ['parcelid']

floats = ['basementsqft', 'bathroomcnt', 'bedroomcnt', 'calculatedbathnbr', 'finishedfloor1squarefeet', 
          'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet13',
          'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'fireplacecnt',
          'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'latitude', 'longitude',
          'lotsizesquarefeet', 'poolcnt', 'poolsizesum', 'roomcnt', 'threequarterbathnbr', 'unitcnt',
          'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt', 'numberofstories',
          'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
          'landtaxvaluedollarcnt', 'taxamount', 'taxdelinquencyyear']

objects = ['airconditioningtypeid', 'architecturalstyletypeid', 'buildingclasstypeid',
           'buildingqualitytypeid', 'decktypeid', 'fips', 'hashottuborspa', 'heatingorsystemtypeid',
           'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'propertycountylandusecode',
           'propertylandusetypeid', 'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
           'regionidcounty', 'regionidneighborhood', 'regionidzip', 'storytypeid',
           'typeconstructiontypeid', 'fireplaceflag', 'taxdelinquencyflag', 'censustractandblock']

feature_dtypes = {col: col_type for type_list, col_type in zip([ints, floats, objects],
                                                               ['int64', 'float64', 'object']) 
                                  for col in type_list}

In [3]:
train_2017 = pd.read_csv("train_2017.csv")
train_2016 = pd.read_csv("train_2016.csv")
properties_2017 = pd.read_csv("properties_2017.csv", dtype = feature_dtypes)
properties_2016 = pd.read_csv("properties_2016.csv", dtype = feature_dtypes)

### Percentage of missing value

In [4]:
dat_2017 = pd.merge(train_2017, properties_2017, on = 'parcelid', how = 'left')

In [5]:
dat_2017.shape

(77613, 60)

In [6]:
#missingValueColumns = dat_2017.columns[dat_2017.isnull().any()].tolist()

In [7]:
# msno.bar(dat_2017[missingValueColumns],\
#             figsize=(20,8),color="#34495e",fontsize=12,labels=True)

In [8]:
mis_perc = pd.DataFrame(dat_2017.isnull().sum()/len(dat_2017))
mis_perc.columns = ['percent']

In [9]:
mis_perc.sort_values(by='percent', ascending = False)

,percent
buildingclasstypeid,0.999807
finishedsquarefeet13,0.999459
basementsqft,0.999356
storytypeid,0.999356
yardbuildingsqft26,0.999098
fireplaceflag,0.997784
architecturalstyletypeid,0.997333
typeconstructiontypeid,0.997127
finishedsquarefeet6,0.995027
pooltypeid10,0.994009


# Handling missing value

In [10]:
dat_2017.drop(['buildingclasstypeid', 'finishedsquarefeet13', 'basementsqft', 'storytypeid',\
                          'fireplaceflag', 'architecturalstyletypeid', 'typeconstructiontypeid', 'finishedsquarefeet6',\
                          'poolsizesum', 'hashottuborspa', 'taxdelinquencyyear', 'finishedsquarefeet15',\
                          'finishedfloor1squarefeet', 'finishedsquarefeet50', 'threequarterbathnbr',\
                          'poolcnt', 'censustractandblock', 'rawcensustractandblock'], axis = 1, inplace = True)

In [11]:
drop_row = ['structuretaxvaluedollarcnt','regionidzip','taxamount','landtaxvaluedollarcnt','taxvaluedollarcnt','bathroomcnt','assessmentyear','bedroomcnt','regionidcounty','fips','latitude','longitude','propertycountylandusecode','propertylandusetypeid','roomcnt', 'yearbuilt']
for row in drop_row:
    dat_2017 = dat_2017[dat_2017[row].notnull()]

In [12]:
rep01 = ['pooltypeid10','pooltypeid2','pooltypeid7']
for col in rep01:
    dat_2017[col].fillna(0, inplace = True)
    for i in range(len(dat_2017[col])):
        if dat_2017.loc[dat_2017.index[i],col] == '1':
            dat_2017.loc[dat_2017.index[i],col] = 1

In [13]:
rep0 = ['yardbuildingsqft26','yardbuildingsqft17','fireplacecnt','garagecarcnt','garagetotalsqft']
for col in rep0:
    dat_2017[col].fillna(0, inplace = True)


In [14]:
dat_2017['decktypeid'].fillna(0, inplace = True)
for i in range(len(dat_2017['decktypeid'])):
        if dat_2017.loc[dat_2017.index[i],'decktypeid'] == '66':
            dat_2017.loc[dat_2017.index[i],'decktypeid'] = 1

In [15]:
dat_2017['taxdelinquencyflag'].fillna(0, inplace = True)
for i in range(len(dat_2017['taxdelinquencyflag'])):
        if dat_2017.loc[dat_2017.index[i],'taxdelinquencyflag'] == 'Y':
            dat_2017.loc[dat_2017.index[i],'taxdelinquencyflag'] = 1

In [16]:
dat_2017['numberofstories'].fillna(0, inplace = True)
for i in range(len(dat_2017['numberofstories'])):
        if dat_2017.loc[dat_2017.index[i],'numberofstories'] == '7':
            dat_2017.loc[dat_2017.index[i],'numberofstories'] = 1

In [17]:
dat_2017.numberofstories.unique()

array([ 0.,  1.,  2.,  3.,  6.])

In [18]:
rep_mis = ['airconditioningtypeid','regionidneighborhood','heatingorsystemtypeid',\
           'propertyzoningdesc','unitcnt','regionidcity']
for col in rep_mis:
    dat_2017[col].fillna('mis', inplace = True)

In [19]:
for i in range(len(dat_2017.buildingqualitytypeid)):
#     print(type(dat_2017.loc[dat_2017.index[i], 'buildingqualitytypeid']))
    if type(dat_2017.loc[dat_2017.index[i], 'buildingqualitytypeid']) is str:
        dat_2017.loc[dat_2017.index[i], 'buildingqualitytypeid'] = float(dat_2017.loc[dat_2017.index[i], 'buildingqualitytypeid']) 

In [20]:
#dat_2017.to_csv('2017dat.csv')

# Additional steps for OLS(fill na with mean)

In [21]:
rep_mean = ['buildingqualitytypeid','lotsizesquarefeet','finishedsquarefeet12',\
            'fullbathcnt','calculatedbathnbr','calculatedfinishedsquarefeet']
for col in rep_mean:
    dat_2017[col].fillna(dat_2017[col].mean(), inplace = True)


In [22]:
dat_2017.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,...,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag
0,14297519,0.025595,2017-01-01,mis,3.5,4.0,6.536347,3.5,0,3100.0,...,0.0,0.0,1998.0,0.0,485713.0,1023282.0,2016.0,537569.0,11013.72,0
1,17052889,0.055619,2017-01-01,mis,1.0,2.0,6.536347,1.0,0,1465.0,...,0.0,0.0,1967.0,1.0,88000.0,464000.0,2016.0,376000.0,5672.48,0
2,14186244,0.005383,2017-01-01,mis,2.0,3.0,6.536347,2.0,0,1243.0,...,0.0,0.0,1962.0,1.0,85289.0,564778.0,2016.0,479489.0,6488.30,0
3,12177905,-0.103410,2017-01-01,mis,3.0,4.0,8.000000,3.0,0,2376.0,...,0.0,0.0,1970.0,0.0,108918.0,145143.0,2016.0,36225.0,1777.51,0
4,10887214,0.006940,2017-01-01,1,3.0,3.0,8.000000,3.0,0,1312.0,...,0.0,0.0,1964.0,0.0,73681.0,119407.0,2016.0,45726.0,1533.89,0


In [23]:
dat_2017.isnull().any()

parcelid                        False
logerror                        False
transactiondate                 False
airconditioningtypeid           False
bathroomcnt                     False
bedroomcnt                      False
buildingqualitytypeid           False
calculatedbathnbr               False
decktypeid                      False
calculatedfinishedsquarefeet    False
finishedsquarefeet12            False
fips                            False
fireplacecnt                    False
fullbathcnt                     False
garagecarcnt                    False
garagetotalsqft                 False
heatingorsystemtypeid           False
latitude                        False
longitude                       False
lotsizesquarefeet               False
pooltypeid10                    False
pooltypeid2                     False
pooltypeid7                     False
propertycountylandusecode       False
propertylandusetypeid           False
propertyzoningdesc              False
regionidcity

In [24]:
dat_2017.to_csv('ols.csv')